In [2]:
import pandas as pd
import math

In [47]:
def zone_detection(x,y):
    distance = math.hypot(x, y)
    # we define three zones with 
    # code 0 for '2PT'
    # code 1 for 'C3'
    # code 2 for 'NC3'
    if abs(y) <= 7.8 and abs(x) >= 22:
        zone = 1 # 'C3'
    elif y > 7.8 and distance >= 23.75:
        zone = 2 # 'NC3'
    else:
        zone = 0 # '2PT'
    return zone

def shot_distribution(shot_list):
    points_total = sum(shot_list)
    shot_dict = [round(shot_list[0]/points_total,3), round(shot_list[1]/points_total,3), round(shot_list[2]/points_total,3)]
    return shot_dict

def calculate_efg(shot_attempt, shot_made):
    # eFG% = (FGM + 0.5*3PM)/FGA
    return round((sum(shot_made) + (0.5*sum(shot_made[1:])))/sum(shot_attempt),3)

def calculate_efg_zone(shot_attempt, shot_made):
    # eFG% = (FGM + 0.5*3PM)/FGA 
    # For 2PT
    efg_2pt = round(shot_made[0]/shot_attempt[0],3)    
    # For C3
    efg_c3 = round((shot_made[1] + (0.5*(shot_made[1])))/shot_attempt[1],3)
    # For NC3
    efg_nc3 = round((shot_made[2] + (0.5*(shot_made[2])))/shot_attempt[2],3)

    return [efg_2pt, efg_c3, efg_nc3]

In [76]:
def final_okc_solution(filename):
    shot_data = pd.read_csv(filename)

    # Each entry [0,0,0] refers to shot attempts of 2PT, C3, NC3 respectively. First element corresponds to Team A and 
    # second element corresponds to Team B
    shot_attempt = [[0,0,0],[0,0,0]]

    # Each entry [0,0,0] refers to successful shot made of 2PT, C3, NC3 respectively. First element corresponds to Team A and 
    # second element corresponds to Team B
    shot_made = [[0,0,0],[0,0,0]]

    for index, row in shot_data.iterrows():    
        if row[0] == 'Team A':
            ind = 0
        else:
            ind = 1
        temp_zone = zone_detection(row[1],row[2])
        shot_attempt[ind][temp_zone] = shot_attempt[ind][temp_zone] + 1
        shot_made[ind][temp_zone] = shot_made[ind][temp_zone] + row[3]

    teams = shot_data['team'].unique()
    final_list = []
    for i in range(0,len(teams)):
        final_list.append([teams[i]])
        final_list[i]+=shot_attempt[i] + shot_distribution(shot_attempt[i]) + shot_made[i] + calculate_efg_zone(shot_attempt[i],shot_made[i])

    new_df = pd.DataFrame(final_list, columns = ['Team', '2PT Attempts', 'C3 Attempts', 'NC3 Attempts', '2PT Shot%', 'C3 Shot%', 'NC3 Shot%', '2PT Made', 'C3 Made', 'NC3 Made', '2PT eFG', 'C3 eFG', 'NC3 eFG'])
    return new_df

final_okc_solution('shots_data.csv')

,Team,2PT Attempts,C3 Attempts,NC3 Attempts,2PT Shot%,C3 Shot%,NC3 Shot%,2PT Made,C3 Made,NC3 Made,2PT eFG,C3 eFG,NC3 eFG
0,Team A,170,20,90,0.607,0.071,0.321,68,8,29,0.40,0.600,0.483
1,Team B,163,21,96,0.582,0.075,0.343,75,5,35,0.46,0.357,0.547
